In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Load both left and right eye dataset
base_path = 'C:/Users/manik/DL-Project/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')


In [3]:
left_df.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_left.jpg,cataract,0,0,0,1,0,0,0,0
1,1,1_left.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_left.jpg,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [4]:
right_df.head()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_right.jpg,normal fundus,1,0,0,0,0,0,0,0
1,1,1_right.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3,3,3_right.jpg,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,4_right.jpg,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [5]:
df = pd.concat([left_df, right_df], axis=0)
df.head()


,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,0_left.jpg,cataract,0,0,0,1,0,0,0,0
1,1,1_left.jpg,normal fundus,1,0,0,0,0,0,0,0
2,2,2_left.jpg,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,3_left.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4,4_left.jpg,macular epiretinal membrane,0,0,0,0,0,0,0,1


In [6]:
df.tail()

,Unnamed: 0,Fundus,Diagnostic Keywords,N,D,G,C,A,H,M,O
3478,3495,4686_right.jpg,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3479,3496,4688_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3480,3497,4689_right.jpg,normal fundus,1,0,0,0,0,0,0,0
3481,3498,4690_right.jpg,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0
3482,3499,4784_right.jpg,hypertensive retinopathy，age-related macular d...,0,0,0,0,1,1,0,0


In [7]:
df.shape


(6968, 11)

In [8]:
# Input pipeline
base_path = 'D:/cropped_training_images/'
image_name = base_path + df['Fundus']
image_name[:5]

0    D:/cropped_training_images/0_left.jpg
1    D:/cropped_training_images/1_left.jpg
2    D:/cropped_training_images/2_left.jpg
3    D:/cropped_training_images/3_left.jpg
4    D:/cropped_training_images/4_left.jpg
Name: Fundus, dtype: object

In [9]:
labels = df.loc[:,['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']]
labels = labels.values

In [10]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [11]:
labels.shape

(6968, 8)

In [12]:
labels = tf.constant(labels, dtype=tf.float32)

In [13]:
image_name

0           D:/cropped_training_images/0_left.jpg
1           D:/cropped_training_images/1_left.jpg
2           D:/cropped_training_images/2_left.jpg
3           D:/cropped_training_images/3_left.jpg
4           D:/cropped_training_images/4_left.jpg
                          ...                    
3478    D:/cropped_training_images/4686_right.jpg
3479    D:/cropped_training_images/4688_right.jpg
3480    D:/cropped_training_images/4689_right.jpg
3481    D:/cropped_training_images/4690_right.jpg
3482    D:/cropped_training_images/4784_right.jpg
Name: Fundus, Length: 6968, dtype: object

In [14]:
labels

<tf.Tensor: shape=(6968, 8), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)>

In [15]:
# Create a function to load the images using TensorFlow's `tf.io.read_file` function and `tf.image.decode_jpeg` function.
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [250, 250])
    image /= 255.0  # normalize the pixels
#     image = tf.expand_dims(image, axis=0) # Add a batch dimension to the image
#     label = tf.transpose(label)
    return image, label

In [16]:
# labels.T

In [17]:
# Load the images and their labels into a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((image_name, labels))
dataset = dataset.map(load_image)
dataset = dataset.batch(16)

In [18]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8), dtype=tf.float32, name=None))>

In [19]:
def split_data(dataset, train_split, test_split, val_split):
    # Calculate the size of each split
    dataset_size = dataset.reduce(tf.constant(0, dtype=tf.int64), lambda acc, _: acc + 1).numpy()
    train_size = int(dataset_size * train_split)
    val_size = int(dataset_size * val_split)
    test_size = int(dataset_size * test_split)

    # Shuffle the elements of the dataset randomly
    dataset = dataset.shuffle(buffer_size=dataset_size, seed=42)

    # Split the dataset into train, validation, and test sets
    train_dataset = dataset.take(train_size)
    remaining_dataset = dataset.skip(train_size)
    val_dataset = remaining_dataset.take(val_size)
    test_dataset = remaining_dataset.skip(val_size)
    return train_dataset, test_dataset, val_dataset

In [20]:
train_dataset, test_dataset, val_dataset = split_data(dataset, train_split=0.7, test_split=0.15, val_split=0.15)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [21]:
from keras.applications import VGG19
base_model = VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(250,250,3)
)
base_model.trainable = False

In [22]:
# # Define the model architecture
# model = tf.keras.Sequential([
#     base_model,
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation='relu'),
#     tf.keras.layers.Dense(8, activation='sigmoid')
# ])

In [23]:
base_model.input

<KerasTensor: shape=(None, 250, 250, 3) dtype=float32 (created by layer 'input_1')>

In [24]:
x = base_model.output
x = tf.keras.layers.Flatten()(x)
fc1 = tf.keras.layers.Dense(units=1024, activation='relu')(x)
output_layer = tf.keras.layers.Dense(units=8, activation='sigmoid')(fc1)

In [25]:
model = tf.keras.Model(inputs=base_model.input, outputs=output_layer)

In [26]:
import tensorflow_addons as tfa
accuracy = tf.keras.metrics.Accuracy()
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
f1_score = tfa.metrics.F1Score(num_classes=8)

In [30]:
# Compile the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
               loss='binary_crossentropy',
              metrics=['accuracy', precision , recall, f1_score])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precision , recall, f1_score])

In [31]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 250, 250, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 250, 250, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 125, 125, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 125, 125, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 125, 125, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 62, 62, 128)       0     

In [32]:
# Train the model
history = model.fit(train_dataset, epochs=10, batch_size=16, validation_data=test_dataset)

Epoch 1/10
305/305 [==============================] - 1297s 4s/step - loss: 0.3831 - accuracy: 0.3838 - precision: 0.4272 - recall: 0.2790 - f1_score: 0.1460 - val_loss: 0.3473 - val_accuracy: 0.4460 - val_precision: 0.4474 - val_recall: 0.4140 - val_f1_score: 0.1055
Epoch 2/10
305/305 [==============================] - 1225s 4s/step - loss: 0.3280 - accuracy: 0.3879 - precision: 0.4190 - recall: 0.1877 - f1_score: 0.2091 - val_loss: 0.2829 - val_accuracy: 0.5248 - val_precision: 0.5409 - val_recall: 0.4947 - val_f1_score: 0.2146
Epoch 3/10
305/305 [==============================] - 1234s 4s/step - loss: 0.3033 - accuracy: 0.4466 - precision: 0.4952 - recall: 0.2371 - f1_score: 0.2476 - val_loss: 0.3013 - val_accuracy: 0.3931 - val_precision: 0.4682 - val_recall: 0.1102 - val_f1_score: 0.2707
Epoch 4/10
305/305 [==============================] - 1299s 4s/step - loss: 0.2968 - accuracy: 0.4477 - precision: 0.4960 - recall: 0.2593 - f1_score: 0.2549 - val_loss: 0.2720 - val_accuracy: 0.5

In [33]:
model.save('C:/Users/manik/DL-Project/DeepLearning/DL-Project/models/model_1_v1.h5')